In [ ]:
#importing required packages
!pip install scanpy -q
import scanpy as sc
import numpy as np
import pandas as pd
!pip install scvi-tools -q
import scvi 
import seaborn as sns
import torch

In [ ]:
from scipy.sparse import csr_matrix
import os

In [ ]:
def pp(filepath):
    #reading the data and transposing it to get genes in row
    adata = sc.read_csv(filepath).T  
    #applying filter and remopving genes that have less than 10 cells
    sc.pp.filter_genes(adata, min_cells = 10)
    #Keeping only the top 2000 most variable genes
    sc.pp.highly_variable_genes(adata, n_top_genes = 2000, subset = True, flavor = 'seurat_v3')
    
    # Specify the device for scvi (GPU if available, otherwise fallback to CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
    scvi.settings.device = device  # Set device for scvi
    
    #model setup and train using scvi
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()
    #doublet detection
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    df = solo.predict()
    df['prediction'] = solo.predict(soft = False)
   
    df['dif'] = df.doublet - df.singlet
    doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
    
    #reloading the data
    adata = sc.read_csv(filepath).T
    adata.obs['Sample'] = filepath.split('_')[1] #'GSM5226574_C51ctr_raw_counts.csv'
    
    #removing any doublets
    adata.obs['doublet'] = adata.obs.index.isin(doublets.index)
    adata = adata[~adata.obs.doublet]
    
    
    sc.pp.filter_cells(adata, min_genes=200) #get rid of cells with fewer than 200 genes
    #sc.pp.filter_genes(adata, min_cells=3) #get rid of genes that are found in fewer than 3 cells
    adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
    #now lets look at ribosomal genes. we are using the preset of ribosomal genes dataframe from broad institute
    ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"
    ribo_genes = pd.read_table(ribo_url, skiprows=2, header = None)


    adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
    adata = adata[adata.obs.n_genes_by_counts < upper_lim]
    adata = adata[adata.obs.pct_counts_mt < 20]
    adata = adata[adata.obs.pct_counts_ribo < 2]


    return adata

In [ ]:
output = []
for file in os.listdir('Data/'):
    output.append(pp('Data/' + file))

In [ ]:
#integrating all the data into one
sc_adata = sc.concat(output)

In [ ]:
sc_adata

In [ ]:
#filtering genes that contain less than 10 cells
sc.pp.filter_genes(sc_adata, min_cells = 100)

In [ ]:
sc_adata.X

In [ ]:
#converting X to dense matrix to reduce storage issues
sc_adata.X = csr_matrix(sc_adata.X)

In [ ]:
#saving it as h5ad(scanpy adata format)
sc_adata.write_h5ad('combined.h5ad')

In [ ]:
adata = sc.read_h5ad('combined.h5ad')
adata

In [ ]:
#looking at how many samples we have
sc_adata.obs

In [ ]:
#saving the raw preprocessed data into layer called counts as it is without performing normalization or log transformation 
sc_adata.layers['counts'] = sc_adata.X.copy()

In [ ]:
#Normalizing the raw datacounts to 10000 in every cell and log tranforming the data
sc.pp.normalize_total(sc_adata, target_sum = 1e4)
sc.pp.log1p(sc_adata)
sc_adata.raw = sc_adata #saving the log normalized data into sc_adata.raw slot

In [ ]:
#viewing the top 
sc_adata

In [ ]:
#setting up anndata model

scvi.model.SCVI.setup_anndata(sc_adata, layer = "counts",
                             categorical_covariate_keys=["Sample"], 
                             continuous_covariate_keys=['pct_counts_mt', 'total_counts', 'pct_counts_ribo'])

In [ ]:
#initializing the model 
model = scvi.model.SCVI(sc_adata)

In [ ]:
#training the initialized model
model.train()

In [ ]:
#after model is trained, we can use get latent representation to get a overlook at our data
#used in single-cell data analysis workflows involving the scVI 
#(single-cell variational inference) model for analyzing single-cell RNA-seq data.
sc_adata.obsm['X_scVI'] = model.get_latent_representation()

In [ ]:
#getting scvi normalization and saving it as another layer scvi_normalized instead of rewriting

sc_adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size = 1e4)

In [ ]:
sc_adata.obsm['X_scVI']

In [ ]:
#Clustering

In [ ]:
#calculating the neighbors using X_scVI
sc.pp.neighbors(sc_adata, use_rep = 'X_scVI')

In [ ]:
#running Umap 
sc.tl.umap(sc_adata)
sc.tl.leiden(sc_adata, resolution = 0.5)

In [ ]:
#plotting umap, one with clusters label and one sample labeled
sc.pl.umap(sc_adata, color = ['leiden', 'Sample'], frameon = False)

In [ ]:
#write the integrated sc_adata object
sc_adata.write_h5ad('integrated.h5ad')

# Find markers/ label cell types

In [ ]:
#changing the resolution to 1
sc.tl.leiden(sc_adata, resolution = 1)

In [ ]:
#getting markers genes based on leiden 
sc.tl.rank_genes_groups(sc_adata, 'leiden')

In [ ]:
#Group = leiden cluster
markers = sc.get.rank_genes_groups_df(sc_adata, None)
#filtering markers with pval_adj less than 0.05 and logFC >.5                                      
markers = markers[(markers.pvals_adj < 0.05) & (markers.logfoldchanges > .5)]
markers

In [ ]:
#using scvi model, calling DE function from our model and grouping by leiden
markers_scvi = model.differential_expression(groupby = 'leiden')
markers_scvi

In [ ]:
#we filter it again: keep only True from "is_de_fdr_0.05" and markers_scvi.lfc mean more than .5
markers_scvi = markers_scvi[(markers_scvi['is_de_fdr_0.05']) & (markers_scvi.lfc_mean > .5)]
markers_scvi

In [ ]:
#create a umap plot 
sc.pl.umap(sc_adata, color = ['leiden'], frameon = False, legend_loc = "on data")

In [ ]:
#for x in range(0,27):
   # print(f'"{x}":","')

In [ ]:
#sc.pl.umap(sc_adata, color = ['PTPRC', 'CD3E', 'CD4'], frameon = False, layer = 'scvi_normalized', vmax = 5)

#sc.pl.umap(sc_adata, color = ['PTPRC', 'CD3E', 'CD8A'], frameon = False, layer = 'scvi_normalized', vmax = 5)

sc.pl.umap(sc_adata, color = ['EPCAM', 'MUC1'], frameon = False, layer = 'scvi_normalized', vmax = 5)


In [ ]:
#markers[markers.names == 'CD8A']

In [ ]:
cell_type = {"0":"Macrophage",
"1":"Fibroblast",
"2":"CD4+ T-cell",
"3":"AT1",
"4":"Macrophage",
"5":"AT2",
"6":"Endothelial cell",
"7":"Plasma cell",
"8":"AT2",
"9":"Macrophage",
"10":"Fibroblast",
"11":"Dendritic cell",
"12":"Fibroblast",
"13":"Cycling T/NK",
"14":"Airway epithelial",
"15":"Airway epithelial",
"16":"Airway epithelial",
"17":"B-cell",
"18":"Aerocyte",
"19":"Airway epithelial",
"20":"Monocyte",
"21":"CD8+ T-cell",
"22":"Neuronal cell",
"23":"Dendritic cell",
"24":"Pericyte",
"25":"Erythroid-like",
"26":"Smooth muscle cell",
"27":"Macrophage"
}

In [ ]:
#markers_scvi[markers_scvi.group1 == '23']

In [ ]:
#for item in markers_scvi[markers_scvi.group1 == '20'] [0:1000].index:
    print(item)

In [ ]:
#mapping using the cell_type dictionary
sc_adata.obs['cell_type'] = sc_adata.obs.leiden.map(cell_type)

In [ ]:
#
sc.pl.umap(sc_adata, color = ['cell_type'], frameon = False)

In [ ]:
sc_adata

In [ ]:
#saving the scvi_markers to the uns data slot
sc_adata.uns['scvi_markers'] = markers_scvi
sc_adata.uns['markers'] = markers

In [ ]:
sc_adata.write_h5ad('integrated.h5ad')

In [ ]:
model.save('model.model')

# Analysis

In [ ]:
import scanpy as sc
import scvi
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:

sc_adata = sc.read_h5ad('integrated.h5ad')

In [ ]:
#listing sample names
sc_adata.obs.Sample.unique().tolist()

In [ ]:
#creating a separate column defining covid sample and control sample
def map_condition(x):
    if 'cov' in x:
        return 'COVID19'
    else:
        return 'control'

In [ ]:
#creating a new column called condition 
sc_adata.obs['condition'] = sc_adata.obs.Sample.map(map_condition)
sc_adata.obs

In [ ]:
#counting the total number of cells in each sample
num_tot_cells = sc_adata.obs.groupby(['Sample']).count()

num_tot_cells = dict(zip(num_tot_cells.index, num_tot_cells.doublet))
num_tot_cells

In [ ]:
#grouping the observation data by sample, condition and cell type
cell_type_counts = sc_adata.obs.groupby(['Sample', 'condition', 'cell type']).count()

cell_type_counts = cell_type_counts[cell_type_counts.sum(axis = 1) > 0].reset_index()
cell_type_counts = cell_type_counts[cell_type_counts.columns[0:4]]
cell_type_counts

In [ ]:
cell_type_counts['total_cells'] = cell_type_counts.Sample.map(num_tot_cells).astype(int)

cell_type_counts['frequency'] = cell_type_counts.doublet / cell_type_counts.total_cells

cell_type_counts

# Plotting

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (10,4))

ax = sns.boxplot(data = cell_type_counts, x = 'cell type', y = 'frequency', hue = 'condition')

plt.xticks(rotation = 35, rotation_mode = 'anchor', ha = 'right')

plt.show()

# Differential expression

In [ ]:
#DE with SCVI

model  = scvi.model.SCVI.load('model.model', sc_adata)
model

In [ ]:
scvi_DE = model.differential_expression(
    idx1 = [sc_adata.obs['cell type'] == 'AT1'],
    idx2 = [sc_adata.obs['cell type'] == 'AT2']
    )


scvi_DE

In [ ]:
#filter out 
scvi_DE = scvi_DE[(scvi_DE['is_de_fdr_0.05']) & (abs(scvi_DE.lfc_mean) > .5)]
scvi_DE = scvi_DE.sort_values('lfc_mean')
scvi_DE

In [ ]:
scvi_DE = scvi_DE[(scvi_DE.raw_normalized_mean1 > .5) | (scvi_DE.raw_normalized_mean2 > .5)]
scvi_DE

In [ ]:
genes_to_show = scvi_DE[-25:].index.tolist() + scvi_DE[:25].index.tolist() #top 25 and bottom 25 from sorted df
sc.pl.heatmap(subset, genes_to_show, groupby='cell_type', swap_axes=True, layer = 'scvi_normalized',
              log = True)

# GO Enrichment

In [ ]:
pip install --upgrade pip

In [ ]:
pip install gseapy -q

In [ ]:
import gseapy as gp 
gp.get_library_name()
# 'GO_Biological_Process_2023',
#'KEGG_2021_Human',

In [ ]:
subset

In [ ]:
enr = gp.enrichr(gene_list= dedf[dedf.log2fc > 0].gene.tolist(),
                 gene_sets=['KEGG_2021_Human','GO_Biological_Process_2023'],
                 organism='human',
                 outdir=None, 
                 background = subset.var_names.tolist()
                )

In [ ]:
enr.results

# comparisons using Violin plot

In [ ]:
sc.pl.violin(subset[subset.obs.cell_type == 'AT2'], 'ETV5', groupby='condition')

In [ ]:
from scipy import stats
temp = subset[subset.obs.cell_type == 'AT2']

i = np.where(temp.var_names == 'ETV5')[0][0]
a = temp[temp.obs.condition == 'COVID19'].X[:,i]
b = temp[temp.obs.condition == 'control'].X[:,i]
stats.mannwhitneyu(a, b)